In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.backends.cudnn as cudnn
import torchmetrics
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import os

In [5]:
transform = transforms.Compose([transforms.ToTensor()])

train_batch_size = 50
test_batch_size = 20
train_set = torchvision.datasets.MNIST(root='./data/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, num_workers=2)

test_set = torchvision.datasets.MNIST(root='./data/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=False, num_workers=2)